In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#Importing the libraries
import json
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
!pip install tensorflow
!pip install nltk
!pip install scikit-learn


In [7]:
# Load intents file
with open('/content/drive/MyDrive/MyBot/intents.json') as file:
    data = json.load(file)

lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

# Preprocess data
for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))


In [8]:
#Preparing Training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))


In [9]:
#Building and training the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Use the legacy SGD optimizer to maintain compatibility
from tensorflow.keras.optimizers.legacy import SGD
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('/content/drive/MyDrive/MyBot/chatbot_model.h5')

Epoch 1/200
41/41 [==============================] - 1s 2ms/step - loss: 3.8686 - accuracy: 0.0392
Epoch 2/200
41/41 [==============================] - 0s 2ms/step - loss: 3.8383 - accuracy: 0.0392
Epoch 3/200
41/41 [==============================] - 0s 2ms/step - loss: 3.8018 - accuracy: 0.0343
Epoch 4/200
41/41 [==============================] - 0s 2ms/step - loss: 3.7512 - accuracy: 0.0784
Epoch 5/200
41/41 [==============================] - 0s 2ms/step - loss: 3.7195 - accuracy: 0.0637
Epoch 6/200
41/41 [==============================] - 0s 2ms/step - loss: 3.6447 - accuracy: 0.0980
Epoch 7/200
41/41 [==============================] - 0s 2ms/step - loss: 3.5592 - accuracy: 0.1373
Epoch 8/200
41/41 [==============================] - 0s 2ms/step - loss: 3.4493 - accuracy: 0.1422
Epoch 9/200
41/41 [==============================] - 0s 2ms/step - loss: 3.2691 - accuracy: 0.1912
Epoch 10/200
41/41 [==============================] - 0s 2ms/step - loss: 3.1744 - accuracy: 0.2010
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
#Adding the chatbot response function
import pickle
from tensorflow.keras.models import load_model

# Save data structures
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Load the model and data structures
model = load_model('/content/drive/MyDrive/MyBot/chatbot_model.h5')
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = get_response(ints, data)
    return res


In [13]:
#Chatting with the bot

num_interactions = 0
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    response = chatbot_response(message)
    print(f"Bot: {response}")
    num_interactions += 1
    if num_interactions >= 5:  # Stop after 5 interactions
        break


def predict_class(sentence, model): # Fixed indentation
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25  # You might need to adjust this threshold
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    # Return a default intent if no intents are above the threshold
    if not return_list:
        return_list.append({"intent": "default", "probability": "0"})

    return return_list

def get_response(intents_list, intents_json): # Fixed indentation
    tag = intents_list[0]['intent']
    # Handle the default intent
    if tag == "default":
        return "I'm not sure I understand. Can you rephrase?"

    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

You: Hello
1/1 [==============================] - 0s 20ms/step
Bot: Greetings! What do you need assistance with?
You: Fever, any advice
1/1 [==============================] - 0s 18ms/step
Bot: To treat a fever at home: 1)Drink plenty of fluids to stay hydrated. 2)Dress in lightweight clothing. 3)Use a light blanket if you feel chilled, until the chills end. 4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others). 5) Get medical help if the fever lasts more than five days in a row.
You: Okay, thanks. what about cuts
1/1 [==============================] - 0s 19ms/step
Bot: Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.
You: Goodbye
1/1 [==============================] - 0s 18ms/step
Bot: Goodbye! Have a great day.
Yo